<a href="https://colab.research.google.com/github/rafaelhamer/DSWP/blob/master/Kaggle_Labdata_Churn_Z1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn.metrics import confusion_matrix # para plotar a confusion matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

def mostra_missing_value(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = 100*round((df.isnull().sum()/df.isnull().count()).sort_values(ascending = False), 2)
    missing_data = pd.concat([total, percent], axis = 1, keys=['Total', 'Percentual'])
    print(missing_data.head(10))

from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import os
for dirname, _, filenames in os.walk('/gdrive/My Drive/Python'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Lendo os dados de Treino e Teste

In [33]:
df_train = pd.read_csv('/gdrive/My Drive/Python/train.csv')#, index_col='id')
df_test  = pd.read_csv('/gdrive/My Drive/Python/test.csv')#, index_col='id')

In [34]:
df_train.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1


In [ ]:
df_train.isna().sum()

In [ ]:
df_train.dtypes

In [5]:
class TrataDataframe():
  def __init__(self,df):
      self.df = df
      self.trata_Pagamentos()
      self.trata_Servicos()
      self.trata_Dependentes()  
      self.trata_MetodoPagamento()

  #df_train.drop('gender', axis=1, inplace=True)

  def trata_Pagamentos(self):
    self.df['TotalCharges'] = self.df['TotalCharges'].str.replace(' ','0.0').astype(np.float64)
    self.df['tenure'] = self.df['tenure'].fillna(self.df['TotalCharges'] / self.df['MonthlyCharges'])
    self.df.drop('TotalCharges', axis=1, inplace=True)
       #self.df['CobrancaTotal'] = pd.to_numeric(self.df['CobrancaTotal'], errors='coerce')
       #self.df.loc[np.isnan(self.df["CobrancaTotal"]),"CobrancaTotal"] = self.df["MesesNaCompanhia"] * self.df["CobrancaMensal"]

  def trata_Servicos(self):
    self.df['MultipleLines'] = self.df['MultipleLines'].replace({'No phone service':'No'})
    self.df['InternetService'] = self.df['InternetService'].replace({'No internet service':'No'})
    self.df['OnlineSecurity'] = self.df['OnlineSecurity'].replace({'No internet service':'No'})
    self.df['OnlineBackup'] = self.df['OnlineBackup'].replace({'No internet service':'No'})
    self.df['DeviceProtection'] = self.df['DeviceProtection'].replace({'No internet service':'No'})
    self.df['TechSupport'] = self.df['TechSupport'].replace({'No internet service':'No'})
    self.df['StreamingTV'] = self.df['StreamingTV'].replace({'No internet service':'No'})
    self.df['StreamingMovies'] = self.df['StreamingMovies'].replace({'No internet service':'No'})

  def trata_Dependentes(self):
    self.df['Dependents'].fillna('No', inplace=True)
        #self.df.loc[self.df["TemDependentes"].isna(),"TemDependentes"] = "MISS"
        #self.df.loc[(self.df["MaisLinhas"] == 'No') & (self.df["TemDependentes"] == 'MISS'), "TemDependentes"] = "No"
        #self.df.loc[(self.df["MaisLinhas"] == 'Yes') & (self.df["TemDependentes"] == 'MISS'), "TemDependentes"] = "Yes"
        #self.df.loc[np.isnan(self.df["TemDependentes"]),"TemDependentes"] = "MISS" #poderia atribuir yes para quem tiver mais de uma linha...

  def trata_MetodoPagamento(self):
    self.df['PaymentMethod'].fillna('MISS', inplace=True)
    #self.df['PaymentMethod'].fillna('Electronic check', inplace=True)
        #self.df = self.df.dropna(axis = 0, subset = ['MetodoPagamento'])
        #self.df.loc[self.df["MetodoPagamento"].isna(),"MetodoPagamento"] = "MISS"

  def getDataFrameDefault(self):
        return self.df  

  def getDataFrameMelhoresFeatures(self):
        ''' retorna o dataframe default, adicionando duas novas colunas (qtdeServicos e vlrMedioPorServico), e igualando "No" e "No internet service" das colunas relativas a serviços online '''
        df2 = self.df.copy()
        df2["QtdeServicosAdicionaisInternet"] = 0
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","QtdeServicosAdicionaisInternet"] = df2["QtdeServicosAdicionaisInternet"] + 1
        df2.loc[df2["ServicoBackupOnline"] == "Yes","QtdeServicosAdicionaisInternet"] =  df2["QtdeServicosAdicionaisInternet"] + 1
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","QtdeServicosAdicionaisInternet"] =  df2["QtdeServicosAdicionaisInternet"] + 1
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","QtdeServicosAdicionaisInternet"] = df2["QtdeServicosAdicionaisInternet"] + 1

        df2.loc[df2["ServicoTV"] != "Yes","ServicoTV"] =  "No"
        df2.loc[df2["ServicoPctFilmes"] != "Yes","ServicoPctFilmes"] = "No"
        df2["QtdeServicosAdicionaisInternet"] = df2["QtdeServicosAdicionaisInternet"].astype('float')
        df2.drop(columns=["genero","TemParceiro","TemTelefone","ServicoSegurancaOnline","ServicoBackupOnline","ServicoProtecaoOnline","ServicoSuporteOnline" ],inplace=True)
        return df2         

  def getDataFramePrecoMedio(self):
        ''' retorna o dataframe default, adicionando duas novas colunas (qtdeServicos e vlrMedioPorServico), e igualando "No" e "No internet service" das colunas relativas a serviços online '''
        df2 = self.df.copy()
        df2["QtdeServicos"] = 0
        df2.loc[df2["TemTelefone"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["TemInternet"] != "No","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoTV"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","QtdeServicos"] = df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoBackupOnline"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoPctFilmes"] == "Yes","QtdeServicos"] = df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","QtdeServicos"] = df2["QtdeServicos"] + 1

        df2.loc[df2["ServicoTV"] != "Yes","ServicoTV"] =  "No"
        df2.loc[df2["ServicoSegurancaOnline"] != "Yes","ServicoSegurancaOnline"] = "No"
        df2.loc[df2["ServicoBackupOnline"] != "Yes","ServicoBackupOnline"] =  "No"
        df2.loc[df2["ServicoProtecaoOnline"] != "Yes","ServicoProtecaoOnline"] = "No"
        df2.loc[df2["ServicoPctFilmes"] != "Yes","ServicoPctFilmes"] = "No"
        df2.loc[df2["ServicoSuporteOnline"] != "Yes","ServicoSuporteOnline"] = "No"
        df2["QtdeServicos"] = df2["QtdeServicos"].astype('float')
        df2.drop(columns=["TemParceiro"],inplace=True)
        #df2["vlrMedioPorServico"] = df2["CobrancaMensal"] / df2["QtdeServicos"]

        return df2  

  def getDataFrameServAdicionais(self):
        ''' gera um dataframe com as transformações "padrão"...e cria uma nova coluna apenas com o indicativo pra saber se há serviços adicionais de internet '''
        df2 = self.df.copy()
        df2["ServicosAdicionaisInternet"] = "No"
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        df2.loc[df2["ServicoBackupOnline"] == "Yes","ServicosAdicionaisInternet"] =  "Yes"
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","ServicosAdicionaisInternet"] =  "Yes"
        df2.loc[df2["ServicoPctFilmes"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        df2.loc[df2["ServicoTV"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        #df2.drop(columns = [ "ServicoSegurancaOnline","ServicoBackupOnline","ServicoSuporteOnline","ServicoProtecaoOnline","ServicoPctFilmes","ServicoTV"],inplace=True)
        df2.drop(columns = ["TemParceiro", "TemDependentes", "ServicoSegurancaOnline","ServicoBackupOnline","ServicoSuporteOnline","ServicoProtecaoOnline","ServicoPctFilmes","ServicoTV"],inplace=True)
        return df2

  def getDataFrameQtdeServicos(self): #80.37
        df2 = self.df.copy()
        df2["QtdeServicos"] = 0
        df2.loc[df2["TemTelefone"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["TemInternet"] != "No","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoTV"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2["QtdeServicos"] = df2["QtdeServicos"].astype('float')
        df2.drop(columns = ["TemTelefone","TemInternet","ServicoTV"],inplace=True)
        return df2          

  def getDataFrameQtdeServicos2(self): 
        df2 = self.df.copy()
        df2["QtdeServicos"] = 0
        df2["QtdeServicosAdicionais"] = 0
        df2.loc[df2["TemTelefone"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["TemInternet"] != "No","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoTV"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoBackupOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoPctFilmes"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2["QtdeServicos"] = df2["QtdeServicos"].astype('float')
        df2["QtdeServicosAdicionais"] = df2["QtdeServicosAdicionais"].astype('float')
        df2.drop(columns = ["TemTelefone","TemInternet","ServicoTV","ServicoSegurancaOnline","ServicoBackupOnline","ServicoSuporteOnline","ServicoProtecaoOnline","ServicoPctFilmes"],inplace=True)
        return df2          


In [47]:
def treina_testa(ts=0.30, it=300, lr=0.03, depth=5, gerarArquivo=False, mostrarFI=False):
   X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(preditoras, target, test_size = ts)#, random_state = i_Seed)
   catb = CatBoostClassifier(iterations=it, learning_rate=lr, depth=depth, silent=True)
   catb_tuned = catb.fit(X_treinamento, y_treinamento, cat_features=categorical_features_indices)
   y_pred = catb_tuned.predict(X_teste)
   acc_catb = round(accuracy_score(y_pred, y_teste) * 100, 2)
   #print('ts: {} it: {} lr: {} depth: {} -> '.format(ts,it,lr,depth)+'Train Data Success Score: %' + str(acc_catb))
   
   if (mostrarFI == True):
      l_fi = list(zip(catb_tuned.feature_importances_,X_treinamento.columns))
      l_fi.sort(reverse=True)
      for tupla in l_fi:
        print(tupla)

   if (gerarArquivo == True):
      df_id = df_testeTratado[["id"]]
      df_teste3 = df_testeTratado.drop(columns=["id"])
      resposta = catb_tuned.predict(df_teste3)
      resposta_df = pd.DataFrame(resposta, columns=['Churn'])
      resultado_submissao = pd.concat([df_id, resposta_df],axis=1)
      resultado_submissao.head().T
      filename = 'submissao_kaggle_catb_fs_ts0{}_it{}_lr{}_depth{}_sc{}.csv'.format(round(ts*100,0),it, lr, depth,str(int(acc_catb*100)))
      resultado_submissao.to_csv(filename, index=False)   
      print(filename)
   
   return acc_catb

In [35]:
df_train = TrataDataframe(df_train).getDataFrameDefault()
df_train.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,4030,Female,0,No,No,56.000000,No,No,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Electronic check,45.05,0
1,6731,Male,0,Yes,Yes,16.928753,Yes,No,No,No,No,No,No,No,No,One year,No,Mailed check,19.65,0
2,6479,Female,0,Yes,No,60.000000,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,0
3,6861,Female,0,No,No,37.000000,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,1
4,3266,Male,0,Yes,Yes,29.000000,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1


In [36]:
df_test = TrataDataframe(df_test).getDataFrameDefault()
df_test.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges
0,5027,Male,0,Yes,Yes,23.0,Yes,No,No,No,No,No,No,No,No,One year,Yes,Credit card (automatic),20.00
1,1733,Male,1,Yes,Yes,61.0,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,No,Electronic check,99.00
2,5384,Male,0,No,No,36.0,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,84.75
3,6554,Female,0,Yes,Yes,61.0,No,No,DSL,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),61.45
4,364,Female,0,No,No,47.0,Yes,No,No,No,No,No,No,No,No,Two year,Yes,Mailed check,20.55


In [26]:
df_train[['Churn','gender']].value_counts()

Churn  gender
0      Male      2094
       Female    2045
1      Male       755
       Female     740
dtype: int64

In [ ]:
df_train[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']].value_counts()

In [ ]:
mostra_missing_value(df_train)

In [ ]:
mostra_missing_value(df_test)

In [51]:
df_train['PaymentMethod'].value_counts()

Electronic check             1986
Mailed check                 1272
Bank transfer (automatic)    1212
Credit card (automatic)      1164
Name: PaymentMethod, dtype: int64

In [49]:
preditoras = df_train.copy()
preditoras.drop(columns=["Churn","id"],inplace=True)
target = df_train["Churn"]
categorical_features_indices = np.where(preditoras.dtypes != np.float)[0] #considerei todas as features que não são do tipo "flutuante" como categóricas
print(f"Qtde de features categóricas: {len(categorical_features_indices)}")
print(f"Colunas preditoras: {preditoras.columns}")

acc = treina_testa(mostrarFI=True)
print(f"acc: {acc}")

Qtde de features categóricas: 16
Colunas preditoras: Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges'],
      dtype='object')
(29.391066633447178, 'Contract')
(18.768888398736426, 'tenure')
(16.33962461513858, 'InternetService')
(9.983454695789037, 'MonthlyCharges')
(5.300840836499578, 'PaymentMethod')
(4.2916165851554124, 'PaperlessBilling')
(3.284891862306668, 'OnlineSecurity')
(2.060142700167638, 'TechSupport')
(1.9250265792008519, 'StreamingMovies')
(1.7403333934946952, 'PhoneService')
(1.4243253243087683, 'SeniorCitizen')
(1.2717783345196425, 'StreamingTV')
(1.1503094454092406, 'Dependents')
(1.0037830158810312, 'MultipleLines')
(0.8397576754574879, 'OnlineBackup')
(0.5716690721409206, 'Partner')
(0.4237537906234347,

## Transformando variáveis categóricas com `LabelEncoder`

In [ ]:
from sklearn.preprocessing import LabelEncoder

for cat_var in df_train.select_dtypes(include='O').columns:
    le = LabelEncoder()
    le.fit(df_train[cat_var])
    df_train[cat_var + '_num'] = le.transform(df_train[cat_var])
    df_train.drop(cat_var, axis=1, inplace=True)
    df_test[cat_var + '_num'] = le.transform(df_test[cat_var])
    df_test.drop(cat_var, axis=1, inplace=True)

df_train.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,4030,Female,0,No,No,56.000000,No,No,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Electronic check,45.05,0
1,6731,Male,0,Yes,Yes,16.928753,Yes,No,No,No,No,No,No,No,No,One year,No,Mailed check,19.65,0
2,6479,Female,0,Yes,No,60.000000,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,0
3,6861,Female,0,No,No,37.000000,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,1
4,3266,Male,0,Yes,Yes,29.000000,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1


In [ ]:
df_test.head()

,SeniorCitizen,tenure,MonthlyCharges,gender_num,Partner_num,Dependents_num,PhoneService_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
id,,,,,,,,,,,,,,,,,,
5027,0,23.0,20.00,1,1,1,1,0,2,0,0,0,0,0,0,1,1,1
1733,1,61.0,99.00,1,1,1,1,1,1,0,1,0,0,1,1,1,0,2
5384,0,36.0,84.75,1,0,0,1,1,1,0,0,0,0,0,1,0,1,2
6554,0,61.0,61.45,0,1,1,0,0,0,0,1,1,1,1,1,2,1,0
364,0,47.0,20.55,0,0,0,1,0,2,0,0,0,0,0,0,2,1,3


## Eliminando variáveis

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
df_train_limpo = df_train.copy()
df_test_limpo = df_test.copy()

In [ ]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(df_train_limpo)

array([[5.60000000e+01, 4.50500000e+01, 2.56010000e+03, ...,
        2.00000000e+00, 0.00000000e+00, 4.00000000e+00],
       [1.69287532e+01, 1.96500000e+01, 3.32650000e+02, ...,
        1.00000000e+00, 0.00000000e+00, 3.00000000e+00],
       [6.00000000e+01, 1.10800000e+02, 6.64070000e+03, ...,
        2.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [2.80000000e+01, 5.46500000e+01, 1.51750000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.30000000e+01, 2.52500000e+01, 1.57305000e+03, ...,
        2.00000000e+00, 0.00000000e+00, 3.00000000e+00],
       [3.80000000e+01, 9.50000000e+01, 3.60560000e+03, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
df_train_limpo

NameError: ignored

In [ ]:
sel

VarianceThreshold(threshold=0.15999999999999998)

## Salvando o dataframe tratado

In [ ]:
df_train.to_csv('/gdrive/My Drive/Python/treino_limpo3.csv')
df_test.to_csv('/gdrive/My Drive/Python/teste_limpo3.csv')